## Importing Packages

In [1]:
!pip install seaborn
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split,cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
!pip install xgboost

In [3]:
from xgboost import XGBClassifier

## Data Pre-processing

In [4]:
fpath = os.path.join(os.getcwd(),'water_dataX.csv')

In [5]:
data = pd.read_csv(fpath,encoding="unicode_escape")
data.head()

,ï»¿STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year
0,1393,"DAMANGANGA AT D/S OF MADHUBAN, DAMAN",DAMAN & DIU,30.6,6.7,7.5,203,NAN,0.1,11,27,2014
1,1399,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOI...,GOA,29.8,5.7,7.2,189,2,0.2,4953,8391,2014
2,1475,ZUARI AT PANCHAWADI,GOA,29.5,6.3,6.9,179,1.7,0.1,3243,5330,2014
3,3181,RIVER ZUARI AT BORIM BRIDGE,GOA,29.7,5.8,6.9,64,3.8,0.5,5382,8443,2014
4,3182,RIVER ZUARI AT MARCAIM JETTY,GOA,29.5,5.8,7.3,83,1.9,0.4,3428,5500,2014


In [6]:
# to see the data types and check for null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991 entries, 0 to 1990
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   ï»¿STATION CODE                   1991 non-null   object
 1   LOCATIONS                         1991 non-null   object
 2   STATE                             1991 non-null   object
 3   Temp                              1991 non-null   object
 4   D.O. (mg/l)                       1991 non-null   object
 5   PH                                1991 non-null   object
 6   CONDUCTIVITY (Âµmhos/cm)          1991 non-null   object
 7   B.O.D. (mg/l)                     1991 non-null   object
 8   NITRATENAN N+ NITRITENANN (mg/l)  1991 non-null   object
 9   FECAL COLIFORM (MPN/100ml)        1991 non-null   object
 10  TOTAL COLIFORM (MPN/100ml)Mean    1991 non-null   object
 11  year                              1991 non-null   int64 
dtypes: int64(1), object(

We did not get any null values because we have all the numerical features as a text



In [7]:
# convert all the columns to numeric except the text column
num_columns = []
for col in data.columns:
    if col not in ['STATION CODE','LOCATIONS','STATE','year']:
        data[col] = pd.to_numeric(data[col],errors='coerce')
        num_columns.append(col)

Numerical columns will be important to identify the water quality index as it should independent of the place and year and should only be depending upon the test results.

Let's check the null values

In [8]:
data.isna().sum()


ï»¿STATION CODE                     122
LOCATIONS                             0
STATE                                 0
Temp                                 92
D.O. (mg/l)                          31
PH                                    8
CONDUCTIVITY (Âµmhos/cm)             25
B.O.D. (mg/l)                        43
NITRATENAN N+ NITRITENANN (mg/l)    225
FECAL COLIFORM (MPN/100ml)          316
TOTAL COLIFORM (MPN/100ml)Mean      132
year                                  0
dtype: int64

In [9]:
# changing the column name
data = data[num_columns[-8:]]
num_columns = ['temp','Disolved oxygen','ph','conductivity','Biochemical oxygen Demand','nitratenan','fecal_coliform','total_coliform']
data.columns = num_columns
data.head()

,temp,Disolved oxygen,ph,conductivity,Biochemical oxygen Demand,nitratenan,fecal_coliform,total_coliform
0,30.6,6.7,7.5,203.0,NaN,0.1,11.0,27.0
1,29.8,5.7,7.2,189.0,2.0,0.2,4953.0,8391.0
2,29.5,6.3,6.9,179.0,1.7,0.1,3243.0,5330.0
3,29.7,5.8,6.9,64.0,3.8,0.5,5382.0,8443.0
4,29.5,5.8,7.3,83.0,1.9,0.4,3428.0,5500.0


We will impute the null values in the data with the median values so that the distribution does not change much

In [10]:
# Null value imputation
imputer = SimpleImputer(strategy='median')
data = pd.DataFrame(imputer.fit_transform(data),columns=data.columns)

In [11]:
# to get the idea how the features are correlated
cor = data.corr()
cor.style.background_gradient(cmap='coolwarm').set_precision(2)

,temp,Disolved oxygen,ph,conductivity,Biochemical oxygen Demand,nitratenan,fecal_coliform,total_coliform
temp,1.00,0.03,0.02,0.14,-0.05,-0.21,0.01,0.01
Disolved oxygen,0.03,1.00,0.03,-0.16,-0.24,-0.20,-0.14,-0.14
ph,0.02,0.03,1.00,-0.02,0.04,-0.01,-0.00,-0.00
conductivity,0.14,-0.16,-0.02,1.00,0.10,0.07,0.00,0.00
Biochemical oxygen Demand,-0.05,-0.24,0.04,0.10,1.00,0.13,0.18,0.16
nitratenan,-0.21,-0.20,-0.01,0.07,0.13,1.00,-0.00,-0.00
fecal_coliform,0.01,-0.14,-0.00,0.00,0.18,-0.00,1.00,0.99
total_coliform,0.01,-0.14,-0.00,0.00,0.16,-0.00,0.99,1.00



From above correlation plot, it is clear that the fecal_coloform and total coliform are highly correlated with each other. So, we are going to remove fecal_coliform as it will be included in total_coliform.

In [12]:
data = data.drop(['fecal_coliform'],axis =1 )

Based on the features we need to calculate the Water Quality Index (WQI) to know if the water is potable or not.

In [13]:
# define some standard parameters
Si = {'Disolved oxygen' : 10,
      'ph' : 8.5,
      'conductivity' : 1000,
      'Biochemical oxygen Demand' : 5,
      'nitratenan' : 45,
      'total_coliform' : 1000
      }
# Ideal values from each parameter
Videal = {'Disolved oxygen' : 14.6,
          'ph' : 7,
          'conductivity' : 0,
          'Biochemical oxygen Demand' : 0,
          'nitratenan' : 0,
          'total_coliform' : 0
          }

# Uniw weights calculations
K = 1/sum([1/i for i in Si.values()])
Wi = {}
for key in Si:
    Wi[key] = K/Si[key]

In [14]:
# to calculate the target variable
def calculate_wqi(data,Si,Wi,Videal):
    df = data.copy()

    for col in Si.keys():
        df[col] = 100 * ((df[col]- Videal[col])/(Si[col] - Videal[col])) * Wi[col]

    return df[Si.keys()].sum(axis = 1)/sum(Wi.values())

In [15]:
data['target'] = calculate_wqi(data,Si,Wi,Videal)

 # classifying the target variable based on reference[1]
target_conditions = [data['target']<=25, # Excellent
                     (data['target']<=50) & (data['target']>=25), # Good
                     (data['target']<=75) & (data['target']>=50), # Poor
                     (data['target']<=100) & (data['target']>=75), # Very poor
                     data['target']>100 # non-drinkable
                     ]
values = [0,1,2,3,4]
data['target'] = np.select(target_conditions,values)

In [16]:
x_train,x_test,y_train,y_test = train_test_split(data.drop('target',axis =1),data['target'],random_state = 42,stratify = data['target'])

References:-


http://article.journalofwaterresources.com/pdf/ajwr-1-3-3.pdf


https://www.mdpi.com/2071-1050/13/8/4259/pdf



